Suggested logic to get goals/points in X previous matches:

- iterate over the list
- get name of HomeTeam
  - start iterating back
  - look for matching name in HomeTeam
  - otherwise, in AwayTeam
  - if match is found, count goals/points
  - count a match, X-1 to go....
- get name of AwayTeam, repeat prev steps

In [187]:
import pandas as pd
import numpy as np
import sys

Let's start by creating a single dataframe for all the seasons I have available

In [188]:
e0_0809 = pd.read_csv('../data/England/E0_0809.csv')
e0_0910 = pd.read_csv('../data/England/E0_0910.csv')
e0_1011 = pd.read_csv('../data/England/E0_1011.csv')
e0_1112 = pd.read_csv('../data/England/E0_1112.csv')
e0_1213 = pd.read_csv('../data/England/E0_1213.csv')
e0_1314 = pd.read_csv('../data/England/E0_1314.csv')
e0_1415 = pd.read_csv('../data/England/E0_1415.csv')

#Let's drop some columns so that the dataframes match exactly and it doesn't screw the concat
e0_0809.drop(['Referee', 'GBH', 'GBD', 'GBA','SBH', 'SBD', 'SBA', 'BSH', 'BSD', 'BSA'], axis=1, inplace=True)
e0_0910.drop(['Referee', 'GBH', 'GBD', 'GBA','SBH', 'SBD', 'SBA', 'BSH', 'BSD', 'BSA'], axis=1, inplace=True)
e0_1011.drop(['Referee', 'GBH', 'GBD', 'GBA','SBH', 'SBD', 'SBA', 'BSH', 'BSD', 'BSA'], axis=1, inplace=True)
e0_1112.drop(['Referee', 'GBH', 'GBD', 'GBA','SBH', 'SBD', 'SBA', 'BSH', 'BSD', 'BSA'], axis=1, inplace=True)
e0_1213.drop(['GBH', 'GBD', 'GBA','PSH', 'PSD', 'PSA', 'BSH', 'BSD', 'BSA'], axis=1, inplace=True)
e0_1314.drop(['Referee','PSH', 'PSD', 'PSA'], axis=1, inplace=True)
e0_1415.drop(['Referee','PSH', 'PSD', 'PSA'], axis=1, inplace=True)

frames = [e0_0809, e0_0910, e0_1011, e0_1112, e0_1213, e0_1314, e0_1415]

df0 = pd.concat(frames, ignore_index=True)
df0.head()
#df0.shape

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA
0,E0,16/08/08,Arsenal,West Brom,1,0,H,1,0,H,...,1.71,1.65,2.25,2.14,26,-1.5,1.88,1.81,2.12,1.97
1,E0,16/08/08,Bolton,Stoke,3,1,H,3,0,H,...,2.33,2.16,1.70,1.64,27,0.0,1.35,1.31,3.56,3.27
2,E0,16/08/08,Everton,Blackburn,2,3,A,1,1,D,...,2.34,2.17,1.70,1.63,28,0.0,1.73,1.43,3.05,2.76
3,E0,16/08/08,Hull,Fulham,2,1,H,1,1,D,...,2.32,2.15,1.69,1.64,26,0.0,1.91,1.85,2.07,1.94
4,E0,16/08/08,Middlesbrough,Tottenham,2,1,H,0,0,D,...,1.90,1.81,2.12,1.92,27,0.0,2.45,2.30,1.65,1.58


Create new DataFrame with the columns I'm interested in-- Team names and full-time results for now

In [189]:
df1 = df0.iloc[:,2:7].copy()
df1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Arsenal,West Brom,1,0,H
1,Bolton,Stoke,3,1,H
2,Everton,Blackburn,2,3,A
3,Hull,Fulham,2,1,H
4,Middlesbrough,Tottenham,2,1,H


In [190]:
df_b365 = df0.loc[:,['B365H','B365D','B365A']]
df_b365.head()

,B365H,B365D,B365A
0,1.20,6.5,15.00
1,1.83,3.5,4.50
2,2.00,3.3,4.00
3,2.60,3.2,2.80
4,3.20,3.4,2.25


Define some parameters-- for now, only the lookback_matches seems relevant

In [191]:
params = {
    'lookback': 7  #number of matches to look back at
}

#define a couple of dicts to translate results keys to points
hpoints = {'H': 3, 'D': 1, 'A': 0}
apoints = {'H': 0, 'D': 1, 'A': 3}

Create new columns to include my new features to trace recent form:

"lookback" Home/Away goals scored, goals conceded, and number of points

In [192]:
def add_lb_columns(df, lookback):
    # Check if quality flag column, i.e., enough matches found to compute lookback results
    # exists..  if not, create it
    if 'LB_FLAG' not in df.columns:
        df['LB_FLAG'] = 1

    #Add columns to df and compute recent form for a given number of lookback games
    df["_".join(["LB", str(lookback), "HGF"])] = 0 #Lookback Home goals for
    df["_".join(["LB", str(lookback), "AGF"])] = 0 #Lookback Away goals for
    df["_".join(["LB", str(lookback), "HGA"])] = 0 #Lookback Home goals against
    df["_".join(["LB", str(lookback), "AGA"])] = 0 #Lookback Away goals against
    df["_".join(["LB", str(lookback), "HP"])] = 0 #Lookback Home points
    df["_".join(["LB", str(lookback), "AP"])] = 0 #Lookback Away points

def add_form(df, lb):

    add_lb_columns(df, lb)
    
    hpoints = {'H': 3, 'D': 1, 'A': 0}
    apoints = {'H': 0, 'D': 1, 'A': 3}

    for index, row in df.iterrows():
        hometeam = row['HomeTeam']
        awayteam = row['AwayTeam']

        #HomeTeam
        lb_hgames = 0  #number of previous games found for home team
        lb_hgf = 0  #cumulative number of goals scored
        lb_hga = 0  #cumulative number of goals conceded
        lb_hpoints = 0  #cumulative number of points
        for j in reversed(xrange(index)):
            if(df.loc[j,'HomeTeam']==hometeam):
                lb_hgf = lb_hgf + df.loc[j,'FTHG']
                lb_hga = lb_hga + df.loc[j,'FTAG']
                lb_hpoints = lb_hpoints + hpoints[df.loc[j,'FTR']]
                lb_hgames+=1
            elif(df.loc[j,'AwayTeam']==hometeam):
                lb_hgf = lb_hgf + df.loc[j,'FTAG']
                lb_hga = lb_hga + df.loc[j,'FTHG']
                lb_hpoints = lb_hpoints + apoints[df.loc[j,'FTR']]
                lb_hgames+=1
            if(lb_hgames==lb):
                break
        df.loc[index, "_".join(["LB", str(lb), "HGF"])] = lb_hgf
        df.loc[index, "_".join(["LB", str(lb), "HGA"])] = lb_hga
        df.loc[index, "_".join(["LB", str(lb), "HP"])] = lb_hpoints

        #AwayTeam
        lb_agames, lb_agf, lb_aga, lb_apoints = 0, 0, 0, 0  #reinitialize lookback vars
        for j in reversed(xrange(index)):
            if(df.loc[j,'HomeTeam']==awayteam):
                lb_agf = lb_agf + df.loc[j,'FTHG']
                lb_aga = lb_aga + df.loc[j,'FTAG']
                lb_apoints = lb_apoints + hpoints[df.loc[j,'FTR']]
                lb_agames+=1
            elif(df.loc[j,'AwayTeam']==awayteam):
                lb_agf = lb_agf + df.loc[j,'FTAG']
                lb_aga = lb_aga + df.loc[j,'FTHG']
                lb_apoints = lb_apoints + apoints[df.loc[j,'FTR']]
                lb_agames+=1
            if(lb_agames==lb):
                break
        df.loc[index, "_".join(["LB", str(lb), "AGF"])] = lb_agf
        df.loc[index, "_".join(["LB", str(lb), "AGA"])] = lb_aga
        df.loc[index, "_".join(["LB", str(lb), "AP"])] = lb_apoints

        if (lb_hgames<lb) | (lb_agames<lb):
            df.loc[index,'LB_FLAG'] = 0
        #else:
        #    df.loc[index,'LB_FLAG'] = 0


In [177]:
add_form(df1,3)
df1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LB_FLAG,LB_3_HGF,LB_3_AGF,LB_3_HGA,LB_3_AGA,LB_3_HP,LB_3_AP
0,Arsenal,West Brom,1,0,H,0,0,0,0,0,0,0
1,Bolton,Stoke,3,1,H,0,0,0,0,0,0,0
2,Everton,Blackburn,2,3,A,0,0,0,0,0,0,0
3,Hull,Fulham,2,1,H,0,0,0,0,0,0,0
4,Middlesbrough,Tottenham,2,1,H,0,0,0,0,0,0,0


In [193]:
add_form(df1,10)
df1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LB_FLAG,LB_10_HGF,LB_10_AGF,LB_10_HGA,LB_10_AGA,LB_10_HP,LB_10_AP
0,Arsenal,West Brom,1,0,H,0,0,0,0,0,0,0
1,Bolton,Stoke,3,1,H,0,0,0,0,0,0,0
2,Everton,Blackburn,2,3,A,0,0,0,0,0,0,0
3,Hull,Fulham,2,1,H,0,0,0,0,0,0,0
4,Middlesbrough,Tottenham,2,1,H,0,0,0,0,0,0,0


Drop rows with not enough lookback games (FLAG = 0)

In [194]:
df1 = df1[df1.LB_FLAG != 0]
df1.drop('LB_FLAG', axis=1, inplace=True)
df1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LB_10_HGF,LB_10_AGF,LB_10_HGA,LB_10_AGA,LB_10_HP,LB_10_AP
99,Chelsea,Sunderland,5,0,H,22,9,4,11,23,12
102,Middlesbrough,West Ham,1,1,D,10,14,15,18,13,12
103,Portsmouth,Wigan,1,2,A,10,11,15,15,14,8
104,Stoke,Arsenal,2,1,H,11,22,18,10,10,20
105,Tottenham,Liverpool,2,1,H,11,15,16,6,6,26


In [195]:
df1.to_csv('../data/England/E0_0815_feats.csv', index=False)

In [196]:
df1.shape

(2441, 11)

In [110]:
df1.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LB_HGF,LB_AGF,LB_HGA,LB_AGA,LB_HP,LB_AP
1895,Hull,Man United,0,0,D,8,16,15,11,8,19
1896,Leicester,QPR,5,1,H,17,13,10,21,20,8
1897,Man City,Southampton,2,0,H,24,16,11,11,21,14
1898,Newcastle,West Ham,2,0,H,6,5,21,12,1,8
1899,Stoke,Liverpool,6,1,H,11,11,10,12,12,14


In [ ]:
df2 = df1[(df1['HomeTeam'].isin(['Arsenal'])) | (df1['AwayTeam'].isin(['Arsenal']))]
#df2.reset_index(drop=True, inplace=True)
#df2.loc[0,'FTHG']=9
df2.head()
#for index, row in df2.iterrows():
#  print index, row['HomeTeam'], row['AwayTeam']
#for i, row in enumerate(df2.values):
#    print row[1]

In [ ]:
team = "Arsenal"
i = 46
df2.loc[i,:].values
lookback = 2
lb_matches = 0
lb_goals = 0

for j in reversed(xrange(i)):
    if(df1.loc[j,"HomeTeam"]==team):
        lb_goals = lb_goals + df1.loc[j,"FTHG"]
        lb_matches+=1
        #print "goals scored by arsenal: %d" %df1.loc[j,"FTHG"]
    elif(df1.loc[j,"AwayTeam"]==team):
        lb_goals = lb_goals + df1.loc[j,"FTAG"]
        lb_matches+=1
        #print "goals scored by arsenal: %d" %df1.loc[j,"FTAG"]
    if(lb_matches==lookback):
        break
    
        #print df1.loc[j,:]
print "Number of goals scored by %s in the last %d matches: %d" %(team, lookback, lb_goals)    

In [ ]:
teams = df1.loc[:,'HomeTeam'].unique()
for team in teams:
    print team

In [7]:
a='lol'
lookback=3

my_list = ["LB", str(lookback), "HGF"]
zz = "_".join(["LB", str(lookback), "HGF"])
print zz

LB_3_HGF
